In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
import pandas as pd
import h5py

In [2]:
# Set main model directory to parent directory
model_dir = os.path.dirname(os.getcwd()) 

In [3]:
# Read Model Scenario Results
scen = h5py.File(model_dir + r'/outputs/daysim_outputs.h5','r+')
scen_name = 'Model: 2040'

In [4]:
# Read Base Data
base_file = r'/inputs/hh_and_persons.h5'

base = h5py.File(model_dir + base_file ,'r+')
base_name = '2006 Survey'

In [5]:

# Results based on Household Location
df_hh = pd.DataFrame({'TAZ': np.asarray(scen['Household']['zone_id']), 
                      'hhsize': np.asarray(scen['Household']['hhsize']),
                      'hhincome': np.asarray(scen['Household']['hhincome']),
                      'hhvehs': np.asarray(scen['Household']['hhvehs']),
                      })

# Results based on Work Location
df_work = pd.DataFrame({'TAZ': np.asarray(scen['Person']['pwtaz']),
                        'transit_pass': np.asarray(scen['Person']['ptpass']),
                        'paid_work_parking': np.asarray(scen['Person']['ppaidprk']),
                        'auto_time_work': np.asarray(scen['Person']['pwautime']),
                        'auto_time_dist': np.asarray(scen['Person']['pwaudist']),
                      })


# Calculate average household size, income, and number of vehicles
df_hh_mean = df_hh.groupby('TAZ').mean()
df_work_mean = df_work.groupby('TAZ').mean()
df_work_mean = df_work_mean.query('TAZ > 0')    # Filter our -1 zone

# Export to csv
df_hh_mean.to_csv('hh_souncast_taz.csv', index=True)
df_work_mean.to_csv('work_souncast_taz.csv', index=True)

In [ ]:
# Create comparison maps, scenario versus some stated base or census/survey data

In [9]:
# Start ArcMap document and map results

RunPath = "."
env.workspace = RunPath
env.qualifiedFieldNames = False

In [128]:
def createTazMap(data, inputlayer, outputdataset, outputlayer):
    ''' Add data to TAZ layer '''
    if arcpy.Exists(outputdataset):
        arcpy.Delete_management(outputdataset)
    if arcpy.Exists(outputlayer):
        arcpy.Delete_management(outputlayer)

    try:
        inputdataset = "TAZ.shp"
        InField = "TAZ"
        JoinField = "TAZ"
        arcpy.MakeFeatureLayer_management(inputdataset, inputlayer)
        arcpy.AddJoin_management(inputlayer, InField, data, JoinField)
        arcpy.CopyFeatures_management(inputlayer, outputdataset)
        arcpy.Delete_management(inputlayer)
    except Exception, e:
        print "Unable to join data"
        print e
        
    # Output the layer as map
    try:
        arcpy.MakeFeatureLayer_management(outputdataset, inputlayer)
        arcpy.SaveToLayerFile_management(inputlayer, outputlayer)
        arcpy.Delete_management(inputlayer)  
    except Exception, e:
        print "Unable to save layer as map"
        print e
        
    return None

In [126]:
# Create a shapefile for household-level results
createTazMap(data='hh_souncast_taz.csv', 
             inputlayer='Household Location', 
             outputdataset='home_loc.shp', 
             'home_loc.lyr')

In [129]:
# Create a shapefile for work-level results
createTazMap(data='work_souncast_taz.csv', 
             inputlayer='Work Location', 
             outputdataset='work_loc.shp', 
             outputlayer='work_loc.lyr')

In [ ]:
###### Try to do all this without writing out to a CSV, reading data in memory ######

In [139]:
out_fc = 'test.gdb/numpy_hh_out'

# Create a numpy array with an id field, and a field with a tuple 
#  of x,y coordinates
# arr = np.array([(1, (471316.3835861763, 5000448.782036674)),
#                    (2, (470402.49348005146, 5000049.216449278))],
#                   np.dtype([('idfield', np.int32),('XY', '<f8', 2)]))

arr = test

# Define a spatial reference for the output feature class
spatial_ref = arcpy.Describe('TAZ.shp').spatialReference

# Export the numpy array to a feature class using the XY field to
#  represent the output point feature
arcpy.da.NumPyArrayToFeatureClass(arr, out_fc, ['XY'], spatial_ref)

TypeError: narray.fields require

In [47]:
a

array([2972, 2989, 2986, ..., 2364, 2365, 2365])

In [138]:
test = np.array([a,np.dtype([('idfield', np.int32),('XY', '<f8', 2)])])

In [140]:
test

array([array([2972, 2989, 2986, ..., 2364, 2365, 2365]),
       dtype([('idfield', '<i4'), ('XY', '<f8', (2,))])], dtype=object)